<a href="https://colab.research.google.com/github/PurpleDin0/698S_BOG/blob/master/698S_Project_BOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#0. git the repo and install all dependancies

In [1]:
!git clone https://github.com/PurpleDin0/698S_BOG.git
%cd 698S_BOG
#%ls
!pip install -r requirements.txt
#once installed make sure to restart the runtime

Cloning into '698S_BOG'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 4), reused 12 (delta 2), pack-reused 0
Unpacking objects: 100% (18/18), done.


     |████████████████████████████████| 235kB 2.9MB/s 
     |████████████████████████████████| 102kB 7.2MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 3.1MB 9.3MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 2.7MB 25.1MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 235kB 51.3MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=f230c6e527444e1049d404bbcbe56d34b2070e5ef67defabef5db2d2c1d10ec6
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built PyDispatcher
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.36.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.22.0 which

#1. Modify the python scripts to target them at the desired websites
Todolist:
 * [] edit the python to replace the targeted site(s) with the desired targets
 * []  

In [0]:
#TODO write a script that removes the generic sites from the imported python code with the desired target(s)
#TODO 

#2. Run the python scriptS

In [2]:
%cd /content/698S_BOG
!python3 sixnineeight.py

Streaming output truncated to the last 5000 lines.
2020-04-03 14:09:12 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/2020/03/24/opinions/prisoners-coronavirus-empathy-jones-reed/index.html 
2020-04-03 14:09:12 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/2020/03/24/opinions/prisoners-coronavirus-empathy-jones-reed/index.html 
2020-04-03 14:09:12 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/2020/03/24/opinions/justice-department-emergency-power-coronavirus-opinion-cross-exam-honig/index.html 
2020-04-03 14:09:12 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/2020/03/24/opinions/justice-department-emergency-power-coronavirus-opinion-cross-exam-honig/index.html 
2020-04-03 14:09:12 [scrapy.spidermiddlewares.depth] DEBUG: Ignoring link (depth > 1): https://www.cnn.com/2015/02/20/us/marco-rubio-fast-facts/index.html 
2020-04-03 

#import data analysis libraries

In [0]:
import pandas as pd

In [4]:
pd.read_json('links.json')

,link
0,https://www.cnn.com/
1,https://www.cnn.com/us
2,https://www.cnn.com/world
3,https://www.cnn.com/politics
4,https://www.cnn.com/business
...,...
4344,https://www.cnn.com/health/video/2020/03/27/v8...
4345,https://www.cnn.com/travel/video/2020/03/27/v8...
4346,https://www.cnn.com/interactive/call-to-earth
4347,https://www.cnn.com/specials/world/freedom-pro...
